In [8]:
import openai
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
df_emb = pd.read_csv('../data/embeddings/df_emb.csv')
embeddings = np.load("../data/embeddings/embeddings.npy")

In [10]:
openai.api_key = ""

In [11]:
n_relevant_chunks = 4

In [12]:
system_prompt = f"""
"""

In [13]:
def semantic_search(query_embedding, embeddings):
    similarities = cosine_similarity([query_embedding], embeddings)[0]
    ranked_indices = np.argsort(-similarities)
    return ranked_indices

def answer_question(chunk, question, model="gpt-3.5-turbo", max_tokens=300, temperature=0.7):
    question = f"Here is some information: {chunk}\nQuestion: {question}."
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "system", "content": system_prompt},
                  {"role": "user", "content": question}],
        max_tokens=max_tokens,
        n=1,
        temperature=temperature,
    )
    return response["choices"][0]["message"]["content"]

def get_embedding(text, model="text-embedding-ada-002"):
    text = text.replace("\n", " ")
    return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

In [17]:
query = "What organizations are you part of"
query_embedding = get_embedding(query)
ranked_indices = semantic_search(np.array(query_embedding), embeddings)
most_relevant_chunk = " ".join(np.array(df_emb['text_chunks'])[ranked_indices[:n_relevant_chunks]].flatten())
answer = answer_question(most_relevant_chunk, query)
answer.strip('\n')

"Well, hello there! My name is Jesse Thaler and I am a Professor of Physics at the Center for Theoretical Physics at MIT. I am also the Director of The NSF AI Institute for Artificial Intelligence and Fundamental Interactions (IAIFI), which is a mouthful, I know. But don't worry, we're not as intimidating as our name suggests. We just like to advance physics knowledge and galvanize AI research innovation, no biggie. \n\nOh, and I am currently serving on the Particle Physics Project Prioritization Panel (P5). You know, just helping to hash out priorities for the future of particle physics for the next 10 years within 20-year context. No pressure or anything. \n\nBut enough about me, let's talk about you! How can I assist you today?"